### Input required: Update information

#### Select columns (exclusive: When start is 1 and end is 5, the applicable range will be 2-4.)
Note: quotation marks should be included. Change values inside of the quotes.

In [1]:
start = 'Q1.2'
end = 'Q3.1'

#### Fill in correct texts for each column.
Note: quotation marks should be included. Change values inside of the quotes.

In [2]:
AY = '2022-2023'
CO = 'c/o 2024'
Clerks = 'Psychiatry'

#### Input a total number of questions/columns per site (for loop purposes). Count all qualitative questions as well.
For example, in this case, we have a total of 43 questions, but will have only 29 quantitiave questions printed in the end.

In [3]:
set_n = 43

That's it. Now you can sit back and run the code! 
#### Initiate

In [4]:
# 1 initiate
from datetime import datetime
import datetime                                               # for timestamp 
import time                                                   # for timers

# start timer
tic = time.time()

import numpy as np
import pandas as pd

#import dataset
readfile = pd.read_excel('input.xlsx', engine='openpyxl')

#create a list of columns
cols = list(readfile.columns)

#subset 16 survey questions
q_start = cols.index(start)
q_end = cols.index(end)
q_subset = readfile.iloc[:,q_start+1:q_end]

#create a list of subset columns
q_cols_subset = cols[q_start+1:q_end]

In [5]:
# 2 create a placeholder dataframe
df0 = pd.DataFrame(columns = ['AcademicYear', 'Class', 'Clerkship'])
#df0

#### Individual sites

In [6]:
# 3 create a dataframe for stats

df1 = pd.DataFrame([])
qn=0
for j in q_cols_subset:

    c = q_subset[j][1:,].value_counts().rename_axis('unique_values').reset_index(name='counts')

    
    if (qn == set_n):
        qn = 1

    else:
        qn +=1
        
    c['QN'] = qn

    cc = {}
    
    for i in range(1,6):
        if i in list(c.unique_values):
            cn = c[c.unique_values == i]['counts']

            cc.update(                {
                    'C'+str(i):int(cn)
                }            )
            
        #filter out invalid values (not a number between 1-5: Text or other coded value for N/A)
        else:
            cc.update(                {
                    'C'+str(i):0
                }              )


    dd = pd.DataFrame([cc])
    dd['N'] = dd.C1 + dd.C2 + dd.C3 + dd.C4 + dd.C5
    dd['Sumproduct'] = dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5
    dd['Mean'] = (dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5)/dd.N
    dd['QN']= c['QN']
    df1 = pd.concat([df1, dd], axis=0, ignore_index=True)
    df1 = df1[['QN','Mean', 'N','Sumproduct']]
#df1

#### Overall weighted means

In [7]:
# 4 create overall stats
df2 = df1.groupby(['QN'], as_index=False).sum()
df2['Mean'] = df2['Sumproduct'] / df2['N']
df2['Question']=df2['QN']
df2 = df2[['Question','Mean', 'N']]
#df2

In [8]:
# 5 create a dataframe for question
#text cell extract

sequence2=0
q0 = []
for l in q_cols_subset:
    q_text_q = str(q_subset[l][0])  
    q0.append(q_text_q)


df3 = pd.DataFrame(q0)
df3.columns = ['Question']
#df3

In [9]:
# 5 concat three dataframes
df4 = pd.concat([df3, df1[['Mean','N']]], axis=1)
df4 = pd.concat([df4,df2], axis=0)
df4= pd.concat([df0,df4], axis=1)
#df4

In [10]:
# 6 delete meaningless qualitative questions
df5 = df4[df4['N'] > 0]

# 7 populate columns for AY, Class, Block
df5 = df5.assign(AcademicYear = AY)
df5 = df5.assign(Class = CO)
df5 = df5.assign(Clerkship = Clerks)
#df5

#### Export

In [11]:
df5.to_excel('output_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")) + '.xlsx', index = False)

In [12]:
# end timer             
toc = time.time() - tic
print('All tasks completed in %.2f s.' % (toc))

All tasks completed in 2.95 s.
